In [36]:
import time
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.model import Model
from sagemaker.tuner import IntegerParameter, CategoricalParameter, \
ContinuousParameter, HyperparameterTuner
from sagemaker.predictor import RealTimePredictor

# Hyperparamaters

In [37]:
num_classes = 2
num_training_samples = 5216
num_layers = 18
mini_batch_size = 128
image_shape = '3,224,224'
augmentation_type = 'crop_color_transform'
epochs = 3
learning_rate = 0.01
use_pretrained_model = 1

In [3]:
job_name_prefix = 'pneumonia-detection'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

In [4]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

input_prefix = 'chest_xray_recordio'
input_train = 's3://{}/{}/train/'.format(bucket, input_prefix)
input_test = 's3://{}/{}/test/'.format(bucket, input_prefix)

In [5]:
output_prefix = 'chest_xray_output'
output_path = 's3://{}/{}/'.format(bucket, output_prefix)

In [16]:
instance_count = 1
instance_type = 'ml.p2.xlarge'
volume_size_gb = 50

In [7]:
train_timeout = 360000

# Get role and training image

In [8]:
role = get_execution_role()
training_image = get_image_uri(boto3.Session().region_name, 
                               'image-classification')

# Create an estimator

In [9]:

estimator = sagemaker.estimator.Estimator(training_image, 
                                          role, 
                                          train_instance_count=instance_count,
                                          train_instance_type=instance_type,
                                          train_volume_size=volume_size_gb,
                                          train_max_run=train_timeout,
                                          output_path=output_path, 
                                          sagemaker_session=sagemaker_session,
                                         input_mode='Pipe')

In [10]:
estimator.set_hyperparameters(num_classes=num_classes,
                              num_training_samples=num_training_samples,
                              num_layers=num_layers,
                              mini_batch_size=mini_batch_size,
                              image_shape=image_shape,
                              augmentation_type=augmentation_type,
                              epochs=epochs,
                              learning_rate=learning_rate,
                              use_pretrained_model=use_pretrained_model)

# Create the training job

In [11]:
s3_input_train = sagemaker.s3_input(s3_data=input_train,
                                    content_type='application/x-recordio')
s3_input_validation = sagemaker.s3_input(s3_data=input_test,
                                        content_type='application/x-recordio')

In [12]:
estimator.fit({
    'train': s3_input_train,
    'validation': s3_input_validation
}, job_name=job_name)

2020-04-06 23:07:11 Starting - Starting the training job...
2020-04-06 23:07:12 Starting - Launching requested ML instances...
2020-04-06 23:08:08 Starting - Preparing the instances for training.........
2020-04-06 23:09:40 Downloading - Downloading input data...
2020-04-06 23:10:00 Training - Downloading the training image......
2020-04-06 23:11:03 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
[04/06/2020 23:11:06 INFO 140030151333696] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[04/06/2020 23:11:06 INFO 140030151333696] M

# Create a tuning job

In [13]:
hyperparameter_ranges = {
    'learning_rate': ContinuousParameter(0.001, 1.0),
    'mini_batch_size': IntegerParameter(64, 128),
    'optimizer': CategoricalParameter(['sgd', 'adam'])
}

objective_metric_name = 'validation:accuracy'
objective_type='Maximize'
max_jobs=6
max_parallel_jobs=2

In [14]:
job_name_prefix = 'bcd-tuning'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

In [17]:
tuner = HyperparameterTuner(estimator=estimator,
                            objective_metric_name=objective_metric_name, 
                            hyperparameter_ranges=hyperparameter_ranges,
                            objective_type=objective_type, 
                            max_jobs=max_jobs, 
                            max_parallel_jobs=max_parallel_jobs)

In [18]:
tuner.fit({
    'train': s3_input_train,
    'validation': s3_input_validation
}, job_name=job_name)
tuner.wait()

........................................................................................!


# Deploy model

In [38]:
role = get_execution_role()
hosting_image = get_image_uri(boto3.Session().region_name, 'image-classification')

In [39]:
role = get_execution_role()
instance_count = 1
instance_type = 'ml.m4.xlarge'

In [40]:
model_name_prefix = 'pnu-image-classification-tuned'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name = model_name_prefix + timestamp

In [25]:
model_artifacts_s3_path = 's3://sagemaker-us-east-2-755441266669/chest_xray_output/bcd-tuning-2020-04-06-23-13-43-002-f524bc13/output/model.tar.gz'
model = Model(
    name=model_name,
    model_data=model_artifacts_s3_path,
    image=hosting_image,
    role=role,
    predictor_cls=lambda endpoint_name, sagemaker_session: RealTimePredictor(endpoint_name, sagemaker_session)
)

In [26]:
endpoint_name_prefix = 'pneumonia-detection-ep'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = endpoint_name_prefix + timestamp

In [27]:
instance_count = 1
instance_type = 'ml.m4.xlarge'

In [ ]:
predictor = model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=instance_count,
    instance_type=instance_type
)

# Testing the deployed model

In [42]:
import glob
import json
import numpy as np
def predict_pneumonia(image_path):
    with open(image_path, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)
    response = predictor.predict(payload)
    result = json.loads(response)
    print('Probabilities for all classes: ', result)
    predicted_class = np.argmax(result)
    if predicted_class == 0:
        print('Pneumonia not detected')
        return 0
    else:
        print('Pneumonia detected')
        return 1

In [43]:
image_path = glob.glob("/home/ec2-user/SageMaker/{0}/val/{1}/*.jpeg".format(
                                                    'chest_xray_standard', 
                                                    'PNEUMONIA'))[0]

In [44]:
predict_pneumonia(image_path)

Probabilities for all classes:  [0.027903562411665916, 0.9720964431762695]
Pneumonia detected


1

In [45]:
image_path = glob.glob("/home/ec2-user/SageMaker/{0}/val/{1}/*.jpeg".format(
                                                    'chest_xray_standard', 
                                                    'NORMAL'))[0]

In [46]:
predict_pneumonia(image_path)

Probabilities for all classes:  [0.9990615248680115, 0.0009384835138916969]
Pneumonia not detected


0

# Evaluation metrics

In [47]:
import pandas as pd
import numpy as np

In [48]:
def evaluation_metrics(confusion_matrix):
    tn, fn, fp, tp = confusion_matrix[0][0], confusion_matrix[0][1], confusion_matrix[1][0], confusion_matrix[1][1]
    accuracy = sum([tn, tp])/sum([tn, fn, fp, tp])
    precision = sum([tp])/ sum([tp, fp])
    recall = sum([tp])/ sum([tp, fn])
    f1_score = 2 * ((precision * recall)/(precision + recall))
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1_score}")
    return accuracy, precision, recall, f1_score

In [49]:
dataset = glob.glob("/home/ec2-user/SageMaker/{0}/val/*/*.jpeg".format('chest_xray_standard'))

In [50]:
predictions = list([ predict_pneumonia(data) for data in dataset ])

Probabilities for all classes:  [0.9990615248680115, 0.0009384835138916969]
Pneumonia not detected
Probabilities for all classes:  [0.9997650980949402, 0.000234856634051539]
Pneumonia not detected
Probabilities for all classes:  [0.9978341460227966, 0.0021658455953001976]
Pneumonia not detected
Probabilities for all classes:  [0.9846579432487488, 0.01534203439950943]
Pneumonia not detected
Probabilities for all classes:  [0.9722853899002075, 0.02771466225385666]
Pneumonia not detected
Probabilities for all classes:  [0.9712350964546204, 0.028764856979250908]
Pneumonia not detected
Probabilities for all classes:  [0.9901774525642395, 0.009822608903050423]
Pneumonia not detected
Probabilities for all classes:  [0.9956036806106567, 0.004396365024149418]
Pneumonia not detected
Probabilities for all classes:  [0.027903562411665916, 0.9720964431762695]
Pneumonia detected
Probabilities for all classes:  [4.5117038098396733e-05, 0.9999549388885498]
Pneumonia detected
Probabilities for all clas

In [58]:
all_image_labels = [ (1 if ('PNEUMONIA' in data) else 0) for data in dataset ]

In [60]:
confusion_matrix = pd.crosstab(index=np.array(all_image_labels), columns=np.array(predictions),
                               rownames=['actual'], colnames=['prediction'])
confusion_matrix

prediction,0,1
actual,,
0,8,0
1,0,8


In [61]:
evaluation_metrics(confusion_matrix)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


(1.0, 1.0, 1.0, 1.0)